### Notebook Overview

This Jupyter notebook is the deliverable for **Course 2 – "Get Started with Python"** in the *Google Advanced Data Analytics Professional Certificate* course. Its goals are to  

1. **Load and document the Waze churn dataset.**  
2. **Compile summary statistics and exploratory insights (Milestone 2).**  
3. **Highlight behavioural differences between retained and churned users** to guide feature engineering and model design in later milestones.  
4. **Conclude with an executive-level summary** of findings and next-step recommendations.
---

#### Data cleanliness

The dataset is nearly production-ready. The only quality issue is **700 missing values in the `label` column** (~ 3 % of rows). Because data cleaning is *not* the focus of this project, we handle it as follows:

- The official course walkthrough **keeps** these unlabeled rows.  
- **This notebook excludes them during analysis** to keep churn vs retention metrics unbiased. Dropping them changes means/medians by ≤ 1 %, so analytical integrity is preserved.  
- The omitted records are stored separately for potential post-model scoring or other experiments.

With that minimal preprocessing complete, we proceed directly to exploratory analysis and finish with the executive summary.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load csv into dataframe
waze_df = pd.read_csv('../data/waze_dataset.csv')
waze_df.head()

,ID,label,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days,device
0,0,retained,283,226,296.748273,2276,208,0,2628.845068,1985.775061,28,19,Android
1,1,retained,133,107,326.896596,1225,19,64,13715.920550,3160.472914,13,11,iPhone
2,2,retained,114,95,135.522926,2651,0,0,3059.148818,1610.735904,14,8,Android
3,3,retained,49,40,67.589221,15,322,7,913.591123,587.196542,7,3,iPhone
4,4,retained,84,68,168.247020,1562,166,5,3950.202008,1219.555924,27,18,Android


In [3]:
# Check for general info, dtypes, etc.
waze_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       14999 non-null  int64  
 1   label                    14299 non-null  object 
 2   sessions                 14999 non-null  int64  
 3   drives                   14999 non-null  int64  
 4   total_sessions           14999 non-null  float64
 5   n_days_after_onboarding  14999 non-null  int64  
 6   total_navigations_fav1   14999 non-null  int64  
 7   total_navigations_fav2   14999 non-null  int64  
 8   driven_km_drives         14999 non-null  float64
 9   duration_minutes_drives  14999 non-null  float64
 10  activity_days            14999 non-null  int64  
 11  driving_days             14999 non-null  int64  
 12  device                   14999 non-null  object 
dtypes: float64(3), int64(8), object(2)
memory usage: 1.5+ MB


In [4]:
# check for null values
waze_df.isna().sum()

ID                           0
label                      700
sessions                     0
drives                       0
total_sessions               0
n_days_after_onboarding      0
total_navigations_fav1       0
total_navigations_fav2       0
driven_km_drives             0
duration_minutes_drives      0
activity_days                0
driving_days                 0
device                       0
dtype: int64

In [5]:
# Check out the rows where 'label' is null
waze_df[waze_df['label'].isna()]

,ID,label,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days,device
77,77,NaN,63,50,133.104155,783,201,0,2649.015822,1517.209970,19,13,iPhone
80,80,NaN,116,93,436.060183,1584,283,62,4183.409514,3121.889952,18,15,iPhone
98,98,NaN,78,64,583.492789,3414,0,0,1811.140893,642.189122,12,11,Android
111,111,NaN,106,102,113.379056,2228,14,0,2817.481840,2011.724274,17,13,Android
142,142,NaN,32,26,222.129310,208,55,10,2459.816477,874.427617,11,7,iPhone
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14941,14941,NaN,191,160,485.328204,1287,25,0,6468.181924,3466.104564,14,14,iPhone
14943,14943,NaN,48,38,96.797017,555,0,6,8266.129497,5902.351711,19,19,iPhone
14945,14945,NaN,34,29,134.416604,1643,268,2,4554.007843,1579.211201,18,17,Android
14972,14972,NaN,220,181,256.212166,1718,360,23,5586.913459,4104.440202,19,18,iPhone


In [6]:
# Compare general stats of the label nulls with non-null labels
waze_unlabeled = waze_df[waze_df['label'].isna()].copy()
waze_unlabeled.describe()

,ID,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days
count,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,7405.584286,80.837143,67.798571,198.483348,1709.295714,118.717143,30.371429,3935.967029,1795.123358,15.382857,12.125714
std,4306.900234,79.987440,65.271926,140.561715,1005.306562,156.308140,46.306984,2443.107121,1419.242246,8.772714,7.626373
min,77.000000,0.000000,0.000000,5.582648,16.000000,0.000000,0.000000,290.119811,66.588493,0.000000,0.000000
25%,3744.500000,23.000000,20.000000,94.056340,869.000000,4.000000,0.000000,2119.344818,779.009271,8.000000,6.000000
50%,7443.000000,56.000000,47.500000,177.255925,1650.500000,62.500000,10.000000,3421.156721,1414.966279,15.000000,12.000000
75%,11007.000000,112.250000,94.000000,266.058022,2508.750000,169.250000,43.000000,5166.097373,2443.955404,23.000000,18.000000
max,14993.000000,556.000000,445.000000,1076.879741,3498.000000,1096.000000,352.000000,15135.391280,9746.253023,31.000000,30.000000


In [7]:
waze_labeled = waze_df.dropna(subset=['label']).copy()
waze_labeled.describe()

,ID,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days
count,14299.000000,14299.000000,14299.000000,14299.000000,14299.000000,14299.000000,14299.000000,14299.000000,14299.000000,14299.000000,14299.000000
mean,7503.573117,80.623820,67.255822,189.547409,1751.822505,121.747395,29.638296,4044.401535,1864.199794,15.544653,12.182530
std,4331.207621,80.736502,65.947295,136.189764,1008.663834,147.713428,45.350890,2504.977970,1448.005047,9.016088,7.833835
min,0.000000,0.000000,0.000000,0.220211,4.000000,0.000000,0.000000,60.441250,18.282082,0.000000,0.000000
25%,3749.500000,23.000000,20.000000,90.457733,878.500000,10.000000,0.000000,2217.319909,840.181344,8.000000,5.000000
50%,7504.000000,56.000000,48.000000,158.718571,1749.000000,71.000000,9.000000,3496.545617,1479.394387,16.000000,12.000000
75%,11257.500000,111.000000,93.000000,253.540450,2627.500000,178.000000,43.000000,5299.972162,2466.928876,23.000000,19.000000
max,14998.000000,743.000000,596.000000,1216.154633,3500.000000,1236.000000,415.000000,21183.401890,15851.727160,31.000000,30.000000


In [8]:
# Comparing summary statistics of the observations with missing retention labels with those that aren't missing any values reveals nothing remarkable.
# The means and standard deviations are fairly consistent between the two groups.

In [9]:
# Get count of null values by device (there are only 2: Android & iPhone)
waze_unlabeled['device'].value_counts()

device
iPhone     447
Android    253
Name: count, dtype: int64

In [ ]:
# Caluculate % of each iPhone nulls & Android nulls
waze_labeled['device'].value_counts(normalize=True) * 100     # normalize=True returns proportions instead of counts

device
iPhone     64.515001
Android    35.484999
Name: proportion, dtype: float64

In [11]:
# Compare with full dataset
waze_df['device'].value_counts(normalize=True) * 100 

device
iPhone     64.484299
Android    35.515701
Name: proportion, dtype: float64

In [12]:
# The percentage of missing values by each device is consistent with their representation in the data overall.
# There is nothing to suggest a non-random cause of the missing data.

In [13]:
# Calculate % of churned vs.retained
waze_labeled['label'].value_counts(normalize=True) * 100

label
retained    82.264494
churned     17.735506
Name: proportion, dtype: float64

In [14]:
# Calculate median values of all columns for churned and retained users
waze_labeled.groupby('label').median(numeric_only=True)

,ID,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days
label,,,,,,,,,,,
churned,7477.5,59.0,50.0,164.339042,1321.0,84.5,11.0,3652.655666,1607.183785,8.0,6.0
retained,7509.0,56.0,47.0,157.586756,1843.0,68.0,9.0,3464.684614,1458.046141,17.0,14.0


In [15]:
# Users who churned averaged ~3 more drives in the last month than retained users, but retained users used the app on over twice as many days as churned users in the same time period.
# The median churned user drove ~200 more kilometers and 2.5 more hours during the last month than the median retained user.
# It seems that churned users had more drives in fewer days, and their trips were farther and longer in duration. Perhaps this is suggestive of a user profile.

In [16]:
# Let's calculate the median kilometers per drive in the last month for both retained and churned users.

# Kilometers per drive; replace 0 drives with NaN to avoid /0 and avoid inf (infinity)
waze_labeled['km_per_drive'] = (
    waze_labeled['driven_km_drives']
    .div(waze_labeled['drives'].replace(0, np.nan))
)

In [17]:
# Calculate the median, and isolate for km per drive
waze_labeled.groupby('label').median(numeric_only=True)[['km_per_drive']]

,km_per_drive
label,
churned,73.491807
retained,74.051037


In [18]:
# Compute km per driving day, avoiding division by zero
waze_labeled['km_per_driving_day'] = (
    waze_labeled['driven_km_drives']
    .div(waze_labeled['driving_days'].replace(0, np.nan))   # NaN -> 0
)

In [19]:
# Calculate the median, and isolate for km per driving day
waze_labeled.groupby('label').median(numeric_only=True)[['km_per_driving_day']]

,km_per_driving_day
label,
churned,523.086749
retained,272.628549


In [20]:
# Calculate the number of drives per driving day for each group, avoiding devision by zero
waze_labeled['drives_per_driving_day'] = (
    waze_labeled['drives']
    .div(waze_labeled['driving_days'].replace(0, np.nan))   # NaN -> 0
)

In [21]:
# Calculate the median number of drives per driving day for each group.
waze_labeled.groupby('label').median(numeric_only=True)[['drives_per_driving_day']]

,drives_per_driving_day
label,
churned,7.454545
retained,3.750000


In [22]:
# See if phone device is a factor for churned vs. retained

# Calculate the number of each device per label
waze_labeled.groupby(['label', 'device']).size()

label     device 
churned   Android     891
          iPhone     1645
retained  Android    4183
          iPhone     7580
dtype: int64

In [23]:
# Calculate the % of each device per label
waze_labeled.groupby('label')['device'].value_counts(normalize=True) * 100

label     device 
churned   iPhone     64.865931
          Android    35.134069
retained  iPhone     64.439344
          Android    35.560656
Name: proportion, dtype: float64

In [24]:
# The ratio of iPhone users and Android users is consistent between the churned group and the retained group,
# and those ratios are both consistent with the ratio found in the overall dataset.

In [25]:
# Create a csv from the cleaner dataset with added features
waze_labeled.to_csv('../data/waze_features_v1.csv', index=False)

In [26]:
# Key insights:
# Churned users log a median 523 km per driving day - almost 2× the distance of retained users — and they also make more trips per day.
# The dataset therefore skews toward 'super-drivers', likely long-haul or commercial operators, rather than typical commuters.
# There is nothing suggestive of churn being correlated with device.

# Recommendation:
# Segment these high-mileage users and collect additional context on their workflows and pain points. 
# Their atypical driving patterns may explain both their heavy use and their churn; tailored features like routing for commercial vehicles or rest-stop alerts could improve retention.

### Executive Summary
[Executive Summary - Milestone 2](../reports/executive_summary_milestone_2.pdf)